In [2]:
import tensorflow as tf
tf.keras.backend.clear_session()


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_dir = "C:/Machine Learning/projek dojo/datasets/augmented"
test_dir = "C:/Machine Learning/projek dojo/datasets/msambare/fer2013/versions/1/test"

train_datagen = ImageDataGenerator(
    rescale=1./255,      
    rotation_range=20,    
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255) 


train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48,48),  
    batch_size=32,
    color_mode="grayscale",  
    class_mode='categorical'  
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48,48),
    batch_size=32,
    color_mode="grayscale",
    class_mode='categorical'
)


Found 100671 images belonging to 3 classes.
Found 7178 images belonging to 7 classes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(7, activation='softmax')  
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


C:\Users\ASUS\anaconda3\envs\mediapipe_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48,48),
    batch_size=32,
    color_mode="grayscale",
    class_mode='categorical'  
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48,48),
    batch_size=32,
    color_mode="grayscale",
    class_mode='categorical'
)


Found 100671 images belonging to 3 classes.
Found 7178 images belonging to 7 classes.


In [6]:
import albumentations as A
import cv2
import numpy as np
import os


augment = A.Compose([
    A.Rotate(limit=20),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=20, p=0.5),
    A.RandomBrightnessContrast(p=0.2)
])


input_folder = "C:/Machine Learning/projek dojo/datasets/msambare/fer2013/versions/1/train"
output_folder = "C:/Machine Learning/projek dojo/datasets/augmented"


for class_name in os.listdir(input_folder):
    class_input_path = os.path.join(input_folder, class_name)
    class_output_path = os.path.join(output_folder, "train", class_name)

    os.makedirs(class_output_path, exist_ok=True)

    for img_name in os.listdir(class_input_path):
        img_path = os.path.join(class_input_path, img_name)
        img = cv2.imread(img_path)

        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            
            augmented = augment(image=img)['image']
            output_path = os.path.join(class_output_path, f"aug_{img_name}")
            cv2.imwrite(output_path, cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR))


C:\Users\ASUS\AppData\Roaming\Python\Python39\site-packages\albumentations\core\validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [7]:
train_data = train_datagen.flow_from_directory(
    "C:/Machine Learning/projek dojo/datasets/augmented/train",
    target_size=(48,48),
    batch_size=32,
    color_mode="grayscale",
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    "C:/Machine Learning/projek dojo/datasets/augmented/test",
    target_size=(48,48),
    batch_size=32,
    color_mode="grayscale",
    class_mode='categorical'
)


Found 57418 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=100,  
    steps_per_epoch=len(train_data),
    validation_steps=len(test_data),
    callbacks=[early_stop]
)


C:\Users\ASUS\anaconda3\envs\mediapipe_env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 257s 143ms/step - accuracy: 0.2478 - loss: 1.8111 - val_accuracy: 0.2827 - val_loss: 1.7317
Epoch 2/100
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 61s 34ms/step - accuracy: 0.2688 - loss: 1.7650 - val_accuracy: 0.3378 - val_loss: 1.6564
Epoch 3/100
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 60s 33ms/step - accuracy: 0.2929 - loss: 1.7262 - val_accuracy: 0.3883 - val_loss: 1.5728
Epoch 4/100
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 62s 34ms/step - accuracy: 0.3145 - loss: 1.6858 - val_accuracy: 0.4139 - val_loss: 1.4955
Epoch 5/100
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 61s 34ms/step - accuracy: 0.3444 - loss: 1.6426 - val_accuracy: 0.4177 - val_loss: 1.4829
Epoch 6/100
1794/1795 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3590 - loss: 1.6202

In [ ]:
model.save("ekspresi_wajah.h5")


In [ ]:
# evaluasi model dengan data test

loss, acc = model.evaluate(test_data)
print(f"Akurasi Model: {acc:.2%}")


NameError: name 'model' is not defined

In [ ]:
loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5429 - loss: 1.2073
Test Loss: 1.2162
Test Accuracy: 0.5358


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Ambil satu batch gambar & label
x_batch, y_batch = next(iter(test_data))

# Prediksi dengan model
predictions = model.predict(x_batch)
predicted_labels = np.argmax(predictions, axis=1)

# Tampilkan beberapa gambar dengan label prediksi
fig, axes = plt.subplots(3, 3, figsize=(10, 10))
class_labels = list(test_data.class_indices.keys())  # Ambil nama kelas

for i, ax in enumerate(axes.flat):
    ax.imshow(x_batch[i].squeeze(), cmap='gray')  # Ubah ke grayscale
    ax.set_title(f"Pred: {class_labels[predicted_labels[i]]}")
    ax.axis("off")

plt.show()


NameError: name 'test_data' is not defined